# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [155]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [156]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [157]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list[1])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [158]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [159]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [160]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    create keyspace if not exists project 
    with replication = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [162]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)
# pandas for verification of rows
df=pd.read_csv('event_datafile_new.csv')
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [163]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# session.execute("drop table table1")
query = "CREATE TABLE IF NOT EXISTS table1 "
query = query + "(artist text, song text, length float,sessionId varchar,itemInSession varchar, PRIMARY KEY (sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [164]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into table1(artist, song,length,sessionID,itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9],float(line[5]),line[8],line[3]))

#### Do a SELECT to verify that the data have been inserted into each table

In [165]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select * from table1 where sessionid='338' and itemInSession='4'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist," | ",row.song," | ",row.length)

Faithless  |  Music Matters (Mark Knight Dub)  |  495.30731201171875


In [166]:
# used pandas for verifying results
df[(df.sessionId==338) & (df.itemInSession==4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [167]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# session.execute("drop table table2")
query = "CREATE TABLE IF NOT EXISTS table2 "
query = query + "(userid int,sessionId int,itemInSession int,\
                artist text,song text,firstname text,lastname text,PRIMARY KEY (userid,Sessionid,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into table2(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))
        
query = "select * from table2 where sessionid=182 and userid=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist," | ",row.song," | ",row.iteminsession," | ",row.firstname," | ",row.lastname)

Down To The Bone  |  Keep On Keepin' On  |  0  |  Sylvie  |  Cruz
Three Drives  |  Greece 2000  |  1  |  Sylvie  |  Cruz
Sebastien Tellier  |  Kilometer  |  2  |  Sylvie  |  Cruz
Lonnie Gordon  |  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  |  3  |  Sylvie  |  Cruz


In [168]:
# used pandas for verifying results
df[(df.sessionId==182) & (df.userId==10)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [169]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 
##                'All Hands Against His Own'
# session.execute("drop table table3")
query = "CREATE TABLE IF NOT EXISTS table3 "
query = query + "(song text,firstname text,lastname text,userid int,PRIMARY KEY (song,firstname,lastname,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into table3(song, firstName, lastName,userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, ( line[9],line[1],line[4],int(line[10])))
        
query = "select * from table3 where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid," | ",row.song," | ",row.firstname," | ",row.lastname)

29  |  All Hands Against His Own  |  Jacqueline  |  Lynch
95  |  All Hands Against His Own  |  Sara  |  Johnson
80  |  All Hands Against His Own  |  Tegan  |  Levine


In [170]:
# used pandas for verifying results
df[df.song=='All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Drop the tables before closing out the sessions

In [139]:
## TO-DO: Drop the table before closing out the sessions
try:
    session.execute("drop table table1")
    session.execute("drop table table2")
    session.execute("drop table table3")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [140]:
session.shutdown()
cluster.shutdown()